In [144]:
import sys
sys.path.append('/Users/manu/GitHub/finsymbols')
import finsymbols
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
import json

start = datetime.datetime(2016,1,1)
end = datetime.date.today()


In [13]:
nasdaq = finsymbols.get_nasdaq_symbols()
nasdaq

[{'company': '1347 Property Insurance Holdings',
  'industry': 'Finance',
  'sector': '2014',
  'symbol': 'PIH'},
 {'company': '1-800 FLOWERS.COM',
  'industry': 'Consumer Services',
  'sector': '1999',
  'symbol': 'FLWS'},
 {'company': '1st Constitution Bancorp (NJ)',
  'industry': 'Savings Institutions',
  'sector': 'Finance',
  'symbol': 'FCCY'},
 {'company': '1st Source Corporation',
  'industry': 'Major Banks',
  'sector': 'Finance',
  'symbol': 'SRCE'},
 {'company': '21Vianet Group',
  'industry': 'Technology',
  'sector': '2011',
  'symbol': 'VNET'},
 {'company': '2U',
  'industry': 'Technology',
  'sector': '2014',
  'symbol': 'TWOU'},
 {'company': '51job',
  'industry': 'Technology',
  'sector': '2004',
  'symbol': 'JOBS'},
 {'company': '8point3 Energy Partners LP',
  'industry': 'Electric Utilities: Central',
  'sector': 'Public Utilities',
  'symbol': 'CAFD'},
 {'company': '8x8 Inc',
  'industry': 'Telecommunications Equipment',
  'sector': 'Public Utilities',
  'symbol': 'E

In [42]:
def ohlc_adj(dat):
    """
    :param dat: pandas DataFrame with stock data, including "Open", "High", "Low", "Close", and "Adj Close", with "Adj Close" containing adjusted closing prices
 
    :return: pandas DataFrame with adjusted stock data
 
    This function adjusts stock data for splits, dividends, etc., returning a data frame with
    "Open", "High", "Low" and "Close" columns. The input DataFrame is similar to that returned
    by pandas Yahoo! Finance API.
    """
    return pd.DataFrame({"Open": dat["Open"] * dat["Adj Close"] / dat["Close"],
                       "High": dat["High"] * dat["Adj Close"] / dat["Close"],
                       "Low": dat["Low"] * dat["Adj Close"] / dat["Close"],
                       "Close": dat["Adj Close"]})

In [58]:
def get_purchase_date():
    start = datetime.datetime(2016,8,1)
    stocks = [['AMD', 6.91, 100], ['DIS', 94.34, 25], ['FB', 126.97, 21], ['IBM', 159.52, 15], 
              ['MSFT', 58.79, 25], ['NVDA', 65.81, 20], ['TSLA', 201.35, 35], ['GOOG', 787.83, 5],
              ['INTC', 37.97, 15]]
    for item in stocks:
        stock = web.DataReader(item[0], "yahoo", start, end)

        idx = (np.abs(stock['Close'].values - item[1])).argmin()
        item.append(stock.index[idx])
        
    return stocks
    
stocks = get_purchase_date()
stocks

[['AMD', 6.91, 100, Timestamp('2016-09-30 00:00:00')],
 ['DIS', 94.34, 25, Timestamp('2016-11-08 00:00:00')],
 ['FB', 126.97, 21, Timestamp('2016-09-09 00:00:00')],
 ['IBM', 159.52, 15, Timestamp('2016-09-01 00:00:00')],
 ['MSFT', 58.79, 25, Timestamp('2016-11-15 00:00:00')],
 ['NVDA', 65.81, 20, Timestamp('2016-10-14 00:00:00')],
 ['TSLA', 201.35, 35, Timestamp('2016-10-12 00:00:00')],
 ['GOOG', 787.83, 5, Timestamp('2016-09-22 00:00:00')],
 ['INTC', 37.97, 15, Timestamp('2016-10-05 00:00:00')]]

In [146]:
start = datetime.datetime(2010,1,1)

def convert_date(x):
    print x

def get_data(symbol, price=None, shares=None, date=None, length=20):
    data = {}
    data['sym'] = symbol
    
    sd = web.DataReader(symbol, "yahoo", start, end)
    sda = ohlc_adj(sd)
    sda["20d"] = np.round(sda["Close"].rolling(window = 20, center = False).mean(), 2)
    sda["50d"] = np.round(sda["Close"].rolling(window = 50, center = False).mean(), 2)
    sda["200d"] = np.round(sda["Close"].rolling(window = 200, center = False).mean(), 2)
    sda["10m"] = sda["Close"] - sda["Close"].shift(10)
    sda["10m"] = np.where((sda["10m"] < 0.01) & (sda["10m"] > -0.01), 0, sda["10m"])
    sda.tail()

    if price:
        profit = (sda['Close'][-1] - price)
        data['pa'] = profit * shares # profit amount
        data['pp'] = ((sda['Close'][-1] / price) - 1)*100 # profit percentage
        data['pd'] = date.strftime('%d-%b-%y')

    data["20d"] = np.around(sda["20d"].values, decimals=3).tolist()[length:]
    data["50d"] = np.around(sda["50d"].values, decimals=3).tolist()[length:]
    data["200d"] = np.around(sda["200d"].values, decimals=3).tolist()[length:]
    data["10m"] = np.around(sda["10m"].values, decimals=3).tolist()[length:]
    data["cls"] = np.around(sda["Close"].values, decimals=3).tolist()[length:]
    data['vol'] = sd['Volume'].values.tolist()
    data['dte'] = [datetime.datetime.strftime(item, '%d-%b-%y') for item in sd.index]
    return data
        
# get_data('FB', 126.97, 21)
# get_data('AMD', 6.91, 100)

adata = []
for item in stocks:
    cdata = get_data(item[0], item[1], item[2], item[3])
    adata.append(cdata)
    
print adata
with open('data.json', 'w') as outfile:
    json.dump(adata, outfile)


[{'pp': 62.083936324167865, 'dte': ['04-Jan-10', '05-Jan-10', '06-Jan-10', '07-Jan-10', '08-Jan-10', '11-Jan-10', '12-Jan-10', '13-Jan-10', '14-Jan-10', '15-Jan-10', '19-Jan-10', '20-Jan-10', '21-Jan-10', '22-Jan-10', '25-Jan-10', '26-Jan-10', '27-Jan-10', '28-Jan-10', '29-Jan-10', '01-Feb-10', '02-Feb-10', '03-Feb-10', '04-Feb-10', '05-Feb-10', '08-Feb-10', '09-Feb-10', '10-Feb-10', '11-Feb-10', '12-Feb-10', '16-Feb-10', '17-Feb-10', '18-Feb-10', '19-Feb-10', '22-Feb-10', '23-Feb-10', '24-Feb-10', '25-Feb-10', '26-Feb-10', '01-Mar-10', '02-Mar-10', '03-Mar-10', '04-Mar-10', '05-Mar-10', '08-Mar-10', '09-Mar-10', '10-Mar-10', '11-Mar-10', '12-Mar-10', '15-Mar-10', '16-Mar-10', '17-Mar-10', '18-Mar-10', '19-Mar-10', '22-Mar-10', '23-Mar-10', '24-Mar-10', '25-Mar-10', '26-Mar-10', '29-Mar-10', '30-Mar-10', '31-Mar-10', '01-Apr-10', '05-Apr-10', '06-Apr-10', '07-Apr-10', '08-Apr-10', '09-Apr-10', '12-Apr-10', '13-Apr-10', '14-Apr-10', '15-Apr-10', '16-Apr-10', '19-Apr-10', '20-Apr-10', '2